<a href="https://colab.research.google.com/github/AkiSugi/sample_Anime2Sketch/blob/main/Anime2Sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに  
Anime2Sketch: A sketch extractor for illustration, anime art, manga  
https://github.com/Mukosame/Anime2Sketch  
2021/5/18 動作確認済み


##事前準備  


In [ ]:
!git clone https://github.com/Mukosame/Anime2Sketch.git

In [ ]:
%cd Anime2Sketch

Pretrained Weights 208M   
https://drive.google.com/drive/folders/1Srf-WYUixK0wiUddc9y3pNKHHno5PN6R  
ダウンロードリンク  
https://drive.google.com/file/d/1RILKwUdjjBBngB17JHwhZNBEaW4Mr-Ml/view?usp=sharing

In [ ]:
import gdown

url='https://drive.google.com/uc?id=1RILKwUdjjBBngB17JHwhZNBEaW4Mr-Ml'
output='./weights/netG.pth'
gdown.download(url, output, quiet=True)

'./weights/netG.pth'

In [ ]:
!wget "https://www.ghibli.jp/gallery/totoro029.jpg" -P test_samples -q
!wget "https://www.ghibli.jp/gallery/porco028.jpg" -P test_samples -q

In [ ]:
from data import get_image_list
from model import create_model
from data import read_img_path, tensor_to_img, save_image

In [ ]:
gpu_ids = [0]
# create model
model = create_model(gpu_ids)      # create a model given opt.model and other options
model.eval()

# 実行


In [ ]:
load_size = 512

### 指定のファイルのみ


In [ ]:
test_path = "test_samples/madoka.jpg"
aus_path = "./res.png"

img,  aus_resize = read_img_path(test_path, load_size)
aus_tensor = model(img)
aus_img = tensor_to_img(aus_tensor)
save_image(aus_img, aus_path, aus_resize)

### 指定のフォルダすべて

In [ ]:
import os

test_list = get_image_list("test_samples")
save_dir = "result"
os.makedirs(save_dir, exist_ok=True)
    
for test_path in test_list:
  basename = os.path.basename(test_path)
  aus_path = os.path.join(save_dir, "sk_"+basename)
  img,  aus_resize = read_img_path(test_path, load_size)
  print(basename)
  print(aus_resize)
  aus_tensor = model(img)
  aus_img = tensor_to_img(aus_tensor)
  save_image(aus_img, aus_path, aus_resize)


# 古いコード  
  GoogleDreiveからのダウンロード  

In [ ]:
# Define some helper functions for downloading pretrained model
# taken from this StackOverflow answer: https://stackoverflow.com/a/39225039

import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
# download and unzip...
file_id = '1RILKwUdjjBBngB17JHwhZNBEaW4Mr-Ml'
destination = './weights/netG.pth'
download_file_from_google_drive(file_id, destination)